# Import Shit

In [28]:
import os
import pandas as pd
from talib import abstract
import numpy as np
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [30]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)


we are going to start with candleStick 

In [2]:
prefix = '/home/yeraf/forex/data/'
pfilename = "/home/yeraf/forex/data/GBPUSD201915M.csv"
savefilename = pfilename[:-4] + "_indicators.csv"
if(not(os.path.isfile(pfilename))):
    print(pfilename + ' not exist')
    exit()

In [3]:
from pandas_datareader import data
import pandas as pd
from datetime import date
from numpy import cov, correlate,corrcoef

start_date = '2015-01-01'
end_date = date.today()

global BTC
BTC = pd.DataFrame(data.DataReader('BTC-USD','yahoo',start_date,end_date)).rename(columns={'High': 'high', 'Low': 'low','Close':'close','Open':'open','Volume':'volume'}).drop(columns={'Adj Close'})

In [4]:
def convertToDateTime(date, time):
    series = []
    for d,t in zip(date, time):
        series.append(str(d) + "|" +t)
    return pd.to_datetime(series, format='%Y%m%d|%H:%M:%S', errors='coerce')

In [5]:
indi = {
    'func' : ['STOCH','sma','sar','MACD','BBANDS','HT_TRENDLINE'],
    'para' : [{'fastk_period': 5,
              'slowk_period': 3,
              'slowk_matype': 0,
              'slowd_period': 3,
              'slowd_matype': 0},
             {
                 'timeperiod': 25
             },
             {
                 'acceleration':0.02,
                 'maximum': 0.2
             },
             {},
             {},
             {}]
}
patternRec = {
    'func': [
        'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE',
        'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS',
        'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY',
        'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLADVANCEBLOCK',
        'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU',
        'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER',
        'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING',
        'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE',
        'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGINGMAN', 'CDLHARAMI',
        'CDLHARAMICROSS', 'CDLHIGHWAVE', 'CDLHIKKAKE', 'CDLHIKKAKEMOD',
        'CDLHOMINGPIGEON', 'CDLIDENTICAL3CROWS', 'CDLINNECK',
        'CDLINVERTEDHAMMER', 'CDLKICKING', 'CDLKICKINGBYLENGTH',
        'CDLLADDERBOTTOM', 'CDLLONGLEGGEDDOJI', 'CDLLONGLINE', 'CDLMARUBOZU',
        'CDLMATCHINGLOW', 'CDLMATHOLD', 'CDLMORNINGDOJISTAR', 'CDLMORNINGSTAR',
        'CDLONNECK', 'CDLPIERCING', 'CDLRICKSHAWMAN', 'CDLRISEFALL3METHODS',
        'CDLSEPARATINGLINES', 'CDLSHOOTINGSTAR', 'CDLSHORTLINE',
        'CDLSPINNINGTOP', 'CDLSTALLEDPATTERN', 'CDLSTICKSANDWICH', 'CDLTAKURI',
        'CDLTASUKIGAP', 'CDLTHRUSTING', 'CDLTRISTAR', 'CDLUNIQUE3RIVER',
        'CDLUPSIDEGAP2CROWS', 'CDLXSIDEGAP3METHODS'
    ],
    'para': []
}
for i in range(0, len(patternRec['func'])):
    patternRec['para'].append(dict())
indi['func'].extend(patternRec['func'])
indi['para'].extend(patternRec['para'])
#defining the rule that we are going to use
rule = {
    'spread': 25,
    'profit': 100,
    'stop_loss': 150
}

In [6]:
def addLag(df, serie, n=1, r=1):
    for i in range (0,n):
        orignal = df[serie]
        modified = orignal.shift((i+1) * r)
        newdf = pd.DataFrame(data= {
            serie + 'lag' + str((i+1) * r) : modified
        })
        df = pd.concat([df,newdf],axis = 1)
    return df

In [7]:
sim = {'tmp':np.arange(100)}
dat = pd.DataFrame(data=sim)
addLag(dat,'tmp',1,-1)

,tmp,tmplag-1
0,0,1.0
1,1,2.0
2,2,3.0
3,3,4.0
4,4,5.0
5,5,6.0
6,6,7.0
7,7,8.0
8,8,9.0
9,9,10.0


In [8]:
def genTA(composed, inputs,indi):
    output = dict()
    print("new composed" + str(composed.shape))
    for i in range(0, len(indi['func'])):
        calculated = np.asarray(abstract.Function(indi['func'][i])(inputs, **(indi['para'][i])))
        for j in range(0,len(abstract.Function(indi['func'][i]).output_names)):
            if len(calculated.shape) == 2:
                output[abstract.Function(indi['func'][i]).output_names[j]] = calculated[j]
            else :
                if abstract.Function(indi['func'][i]).output_names[j] == "real" or abstract.Function(indi['func'][i]).output_names[j] == 'integer':
                    output[abstract.Function(indi['func'][i]).info['name']] = calculated
                else :
                    output[abstract.Function(indi['func'][i]).output_names[j]] = calculated
    TA = pd.DataFrame(data=output).iloc[1:]
    print(TA.shape)
    print(composed.shape)
    return pd.concat([composed,TA],axis = 0)

In [9]:
from hurst import compute_Hc, random_walk
import matplotlib.pyplot as plt
%matplotlib inline  

# Use random_walk() function or generate a random walk series manually:
# series = random_walk(99999, cumprod=True)
np.random.seed(42)
random_changes = 1. + np.random.randn(99999) / 1000.
series = np.cumprod(random_changes)  # create a random walk from random changes

# Evaluate Hurst equation
H, c, data = compute_Hc(BTC['median'], kind='price', simplified=True)

# Plot
f, ax = plt.subplots()
ax.plot(data[0], c*data[0]**H, color="deepskyblue")
ax.scatter(data[0], data[1], color="purple")
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Time interval')
ax.set_ylabel('R/S ratio')
ax.grid(True)
plt.show()

print("H={:.4f}, c={:.4f}".format(H,c))

KeyError: 'median'

In [10]:
def applyRule(x, rule):
    if x['diff'] > 1:
        return 0
    elif x['diff'] < -1:
        return 1
    else:
        return 2
    
    

def calProfit(df,rule):
    LOOKINGFORWARD = 5
    FACTORING = ['high','low','diff']
    for factor in FACTORING:
        df = addLag(df, factor, LOOKINGFORWARD, -1)
    dropList = list()
    print(df.info())
    for factor in FACTORING:
        for i in range (0,LOOKINGFORWARD):
            dropList.append(factor + 'lag' + str((i+1) * -1))
    result = df.progress_apply(lambda x: applyRule(x,rule), axis=1)
    df['signal'] = result
    df = df.drop(dropList,axis = 1)
    return df

In [44]:
def prepDataFrame(pfilename, indicator):
    df = pd.read_csv(pfilename)
    openp = df.loc[1:, 'open'].to_numpy()
    closep = df.loc[1:, 'close'].to_numpy()
    highp = df.loc[1:, 'high'].to_numpy()
    lowp = df.loc[1:, 'low'].to_numpy()
    volumep = df.loc[1:, 'volume'].to_numpy()
    datep = df.loc[1:, 'Date'].tolist()
    timep = df.loc[1:, 'Timestamp'].tolist()
    medianp = (np.asarray(highp) + np.asarray(lowp)) / 2
    del df
    inputs = {
        'open': openp,
        'close': closep,
        'high': highp,
        'low': lowp,
        'volume': volumep
    }
    dat = {
        'time': convertToDateTime(datep, timep),
        'open': openp,
        'close': closep,
        'high': highp,
        'low': lowp,
        'median': medianp,
        'volume': volumep
    }
    composed = pd.DataFrame(data=dat)
    diff = np.diff(composed['close'])
    composed = addLag(composed,'close',1)
    composed = composed.iloc[1:,]
    composed['diff'] = diff
    composed = calProfit(composed, rule)
    composed = genTA(composed, inputs, indicator)
    composed = addLag(composed, 'diff', 5)
    composed = composed.iloc[70:composed.shape[0]-10, :]
    composed = composed.drop(['diff'],axis = 1)
    composed = addLag(composed,'signal',-1)
    composed['signal'] = composed['signallag1']
    composed = composed.drop(['signallag1'],axis = 1)
    return composed

In [16]:
def preProcessing(df, indicator):
    '''
        factor: time,open,close,high,low,median,volume
    '''
    composed = df
    inputs = {
        'open': composed['open'],
        'close': composed['close'],
        'high': composed['high'],
        'low': composed['low'],
        'volume': composed['volume']
    }
    composed['median'] = (np.asarray(composed['high']) + np.asarray(composed['low'])) / 2
    diff = np.diff(composed['close'])
    composed = addLag(composed,'close',1)
    composed = composed.iloc[1:,]
    composed['diff'] = diff
    composed = calProfit(composed, rule)
    composed = genTA(composed, inputs, indicator)
#    composed = addLag(composed, 'diff', 5)
    composed = composed.iloc[70:composed.shape[0]-10, :]
    composed = composed.drop(['diff','closelag1'],axis = 1)
    return composed

In [17]:
from sklearn.utils import shuffle
def saveDataSet(df,train_ratio = 0.80):
    df = shuffle(df)
    trainlen = int(df.shape[0]*train_ratio)
    testlen = df.shape[0] - trainlen
    train = df.head(trainlen+1)
    test = df.tail(testlen)
    trainfile = savefilename[:-4] + 'train_' + str(round(train_ratio,2)) + '.csv'
    testfile = savefilename[:-4] + 'test_' + str(round(1-train_ratio,2)) + '.csv'
    train.to_csv(trainfile)
    test.to_csv(testfile)
    df.to_csv(savefilename[:-4] + "whole.csv")

In [45]:
output = preProcessing(BTC,indi)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1885 entries, 2015-01-02 to 2020-02-29
Data columns (total 23 columns):
high         1885 non-null float64
low          1885 non-null float64
open         1885 non-null float64
close        1885 non-null float64
volume       1885 non-null int64
median       1885 non-null float64
closelag1    1885 non-null float64
diff         1885 non-null float64
highlag-1    1884 non-null float64
highlag-2    1883 non-null float64
highlag-3    1882 non-null float64
highlag-4    1881 non-null float64
highlag-5    1880 non-null float64
lowlag-1     1884 non-null float64
lowlag-2     1883 non-null float64
lowlag-3     1882 non-null float64
lowlag-4     1881 non-null float64
lowlag-5     1880 non-null float64
difflag-1    1884 non-null float64
difflag-2    1883 non-null float64
difflag-3    1882 non-null float64
difflag-4    1881 non-null float64
difflag-5    1880 non-null float64
dtypes: float64(22), int64(1)
memory usage: 433.4 KB
None



new composed(1885, 9)
(1885, 72)
(1885, 9)


/home/yeraf/.pyenv/versions/3.6.2/envs/algotrade/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [46]:
composed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3670 entries, 71 to 1865
Data columns (total 86 columns):
CDL2CROWS              1865 non-null float64
CDL3BLACKCROWS         1865 non-null float64
CDL3INSIDE             1865 non-null float64
CDL3LINESTRIKE         1865 non-null float64
CDL3OUTSIDE            1865 non-null float64
CDL3STARSINSOUTH       1865 non-null float64
CDL3WHITESOLDIERS      1865 non-null float64
CDLABANDONEDBABY       1865 non-null float64
CDLADVANCEBLOCK        1865 non-null float64
CDLBELTHOLD            1865 non-null float64
CDLBREAKAWAY           1865 non-null float64
CDLCLOSINGMARUBOZU     1865 non-null float64
CDLCONCEALBABYSWALL    1865 non-null float64
CDLCOUNTERATTACK       1865 non-null float64
CDLDARKCLOUDCOVER      1865 non-null float64
CDLDOJI                1865 non-null float64
CDLDOJISTAR            1865 non-null float64
CDLDRAGONFLYDOJI       1865 non-null float64
CDLENGULFING           1865 non-null float64
CDLEVENINGDOJISTAR     1865 non-null 

In [42]:
btcfilename = 'btc.csv'
composed = prepDataFrame(btcfilename,indi)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 1 to 1875
Data columns (total 24 columns):
time         0 non-null datetime64[ns]
open         1875 non-null float64
close        1875 non-null float64
high         1875 non-null float64
low          1875 non-null float64
median       1875 non-null float64
volume       1875 non-null int64
closelag1    1875 non-null float64
diff         1875 non-null float64
highlag-1    1874 non-null float64
highlag-2    1873 non-null float64
highlag-3    1872 non-null float64
highlag-4    1871 non-null float64
highlag-5    1870 non-null float64
lowlag-1     1874 non-null float64
lowlag-2     1873 non-null float64
lowlag-3     1872 non-null float64
lowlag-4     1871 non-null float64
lowlag-5     1870 non-null float64
difflag-1    1874 non-null float64
difflag-2    1873 non-null float64
difflag-3    1872 non-null float64
difflag-4    1871 non-null float64
difflag-5    1870 non-null float64
dtypes: datetime64[ns](1), float64(22), int64(1)
me


new composed(1875, 10)
(1875, 72)
(1875, 10)


/home/yeraf/.pyenv/versions/3.6.2/envs/algotrade/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [21]:
unique, counts = np.unique(output['signal'], return_counts=True)
counts

array([930, 767, 118, ...,   1,   1,   1])

In [23]:
output.isnull().sum().max()

1877

In [ ]:
saveDataSet(composed)

In [ ]:
composed.shape

# autoML

In [24]:
from sklearn.model_selection import train_test_split
train=output.sample(frac=0.8,random_state=200) #random state is a seed value
test=output.drop(train.index)
train.fillna(0).to_csv('train.csv')
test.fillna(0).to_csv('test.csv')

In [25]:
import autogluon as ag
from autogluon import TabularPrediction as task
train_data = task.Dataset(file_path='train.csv')
dirr = 'agModels-predictClass' # specifies folder where to store trained models
predictor = task.fit(train_data=train_data, label='signal', output_directory=dirr,problem_type='multiclass')

/home/yeraf/.pyenv/versions/3.6.2/envs/algotrade/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Loaded data from: train.csv | Columns = 80 / 80 | Rows = 2952 -> 2952
Beginning AutoGluon training ...
Preprocessing data ...
	Data preprocessing and feature engineering runtime = 0.26s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
Fitting model: RandomForestClassifierGini ...
	0.99s	 = Training runtime
	0.818	 = Validation accuracy score
Fitting model: RandomForestClassifierEntr ...
	0.9s	 = Training runtime
	0.818	 = Validation accuracy score
Fitting 

In [26]:
test_data = task.Dataset(file_path='test.csv')
y_test = test_data['signal']
test_data_nolabel = test_data.drop(labels=['signal'],axis=1)
y_pred = predictor.predict(test_data_nolabel)
y_pred_proba = predictor.predict_proba(test_data_nolabel)
predictor.leaderboard(test_data)

Loaded data from: test.csv | Columns = 80 / 80 | Rows = 735 -> 735


                         model  score_test  score_val   fit_time  pred_time  stack_level
0         weighted_ensemble_l1    0.888435      0.898   0.558013   0.000487            1
1          NeuralNetClassifier    0.885714      0.870  26.453157   1.057025            0
2           LightGBMClassifier    0.865306      0.876   2.293921   0.028418            0
3     LightGBMClassifierCustom    0.859864      0.868   4.735435   0.021996            0
4           CatboostClassifier    0.857143      0.874  12.492612   0.035180            0
5     ExtraTreesClassifierEntr    0.809524      0.822   0.792560   0.223296            0
6     ExtraTreesClassifierGini    0.808163      0.814   0.799160   0.223187            0
7   RandomForestClassifierEntr    0.802721      0.818   0.898012   0.230646            0
8   RandomForestClassifierGini    0.801361      0.818   0.985267   0.228971            0
9     KNeighborsClassifierDist    0.759184      0.740   0.048687   0.129831            0
10    KNeighborsClass

,model,score_test,score_val,fit_time,pred_time,stack_level
0,weighted_ensemble_l1,0.888435,0.898,0.558013,0.000487,1
1,NeuralNetClassifier,0.885714,0.870,26.453157,1.057025,0
2,LightGBMClassifier,0.865306,0.876,2.293921,0.028418,0
3,LightGBMClassifierCustom,0.859864,0.868,4.735435,0.021996,0
4,CatboostClassifier,0.857143,0.874,12.492612,0.035180,0
5,ExtraTreesClassifierEntr,0.809524,0.822,0.792560,0.223296,0
6,ExtraTreesClassifierGini,0.808163,0.814,0.799160,0.223187,0
7,RandomForestClassifierEntr,0.802721,0.818,0.898012,0.230646,0
8,RandomForestClassifierGini,0.801361,0.818,0.985267,0.228971,0
9,KNeighborsClassifierDist,0.759184,0.740,0.048687,0.129831,0
